In [1]:
import os

In [2]:
%pwd

'd:\\mlops\\datascienceproject\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\mlops\\datascienceproject'

In [7]:
# now first update config.yaml
import pandas as pd

data = pd.read_csv("artifacts\data_ingestion\winequality-red.csv")

In [9]:
data.info

<bound method DataFrame.info of       fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.4             0.700         0.00             1.9      0.076   
1               7.8             0.880         0.00             2.6      0.098   
2               7.8             0.760         0.04             2.3      0.092   
3              11.2             0.280         0.56             1.9      0.075   
4               7.4             0.700         0.00             1.9      0.076   
...             ...               ...          ...             ...        ...   
1594            6.2             0.600         0.08             2.0      0.090   
1595            5.9             0.550         0.10             2.2      0.062   
1596            6.3             0.510         0.13             2.3      0.076   
1597            5.9             0.645         0.12             2.0      0.075   
1598            6.0             0.310         0.47             3.6      0.067

In [10]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [11]:
# goes into config entity
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict

In [12]:
from src.datascience.constants import *
from src.datascience.utils.common import *

In [17]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self)->DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.COLUMNS  

        create_directories([config.root_dir])

        data_validation_config= DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema,
        )
        return data_validation_config

In [18]:
import os
from src.datascience import logger


In [19]:
#now we will define datavalidation

class DataValidation:
    def __init__(self,config:DataValidationConfig):
        self.config = config

    def validate_all_columns(self)->bool:
        try:
            validation_status = None
            data=pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status{validation_status}")
                else:
                    validation_status=True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status{validation_status}")
            return validation_status
        except Exception as e:
            raise e




In [20]:
try:
    config= ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation=DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e  


[2025-01-01 19:27:41,525: INFO: yaml file: config\config.yaml loaded successfully]
[2025-01-01 19:27:41,527: INFO: yaml file: params.yaml loaded successfully]
[2025-01-01 19:27:41,530: INFO: yaml file: schema.yaml loaded successfully]
[2025-01-01 19:27:41,531: INFO: created directory at:artifacts]
[2025-01-01 19:27:41,532: INFO: created directory at:artifacts/data_validation]
